# Langchain Agent with llms.txt (동기 버전)

이 노트북은 **Langchain**과 **Databricks**를 활용하여 llms.txt 기반 Tool-calling Agent를 구현하는 예제입니다.

## 동기(Sync) 버전이란?

> ✅ **Langchain 동기 방식의 표준 예제입니다.** Langchain의 `stream_mode="messages"`를 사용하여 동기 방식으로 토큰 단위 스트리밍을 구현합니다. 비동기 처리가 필요 없는 환경에서 더 간단하게 사용할 수 있습니다.

## llms.txt 접근 방식

이 노트북은 MCP 대신 **llms.txt** 파일을 활용하는 경량 접근 방식을 사용합니다:
- MCP보다 가볍고 설정이 간단
- RAG보다 정확한 문서 참조
- llms.txt를 인덱스로 사용하여 필요한 문서를 동적으로 가져옴

## 이 노트북에서 다루는 내용

- Langchain + Databricks Foundation Model API를 활용한 Agent 구현
- llms.txt 기반 문서 검색 및 페칭 도구 구현
- MLflow의 `ResponsesAgent`를 사용한 래퍼 클래스 작성
- Agent 출력 테스트 및 동기 스트리밍 처리
- Mosaic AI Agent Evaluation을 통한 평가
- Unity Catalog에 모델 등록 및 배포

## 실행 환경

- Databricks Serverless 또는 DBR 17 이상 클러스터
- Unity Catalog 활성화 필수

## 사전 요구사항

- 이 노트북의 모든 `TODO` 항목을 확인하고 수정하세요.

In [0]:
%pip install -U -qqqq backoff databricks-langchain uv databricks-agents mlflow-skinny[databricks]
dbutils.library.restartPython()

## Agent 코드 정의

아래에서 Agent 코드를 단일 셀에 정의합니다. `%%writefile` 매직 명령어를 사용하여 로컬 Python 파일로 저장하면, 이후 MLflow 로깅 및 배포 시 활용할 수 있습니다.

### 주요 구성 요소
- **LLMSTxtLoader**: llms.txt 파일 로드 및 인덱스 파싱
- **DocumentFetcher**: URL에서 문서 내용을 가져오는 유틸리티
- **Tool 정의**: search_docs, fetch_document, get_doc_index
- **ChatDatabricks**: Databricks Foundation Model API 연동

Agent에 추가할 수 있는 도구 예제는 [공식 문서](https://docs.databricks.com/generative-ai/agent-framework/agent-tool.html)를 참고하세요.

In [0]:
%%writefile langchain_sync_agent.py
"""
llms.txt 기반 Langchain Agent (동기 버전)

MCP보다 가볍고 RAG보다 정확한 접근법:
1. llms.txt를 인덱스로 사용 (시작 시 자동 로드)
2. fetch_document tool로 필요한 상세 문서를 동적으로 가져옴
3. 링크 안의 링크도 추적 가능

이 모듈은 Langchain의 동기 방식 표준 예제입니다.
"""

import re
import httpx
from typing import Optional, Any
from functools import lru_cache
from urllib.parse import urljoin

from databricks_langchain import ChatDatabricks
from langchain.agents import create_agent
from langchain_core.tools import tool


# ==============================================================================
# llms.txt 로더 및 파서
# ==============================================================================

class LLMSTxtLoader:
    """llms.txt 파일을 로드하고 파싱하는 클래스"""
    
    def __init__(self, llms_txt_url: str):
        self.llms_txt_url = llms_txt_url
        self._content: Optional[str] = None
        self._index: dict[str, list[dict[str, str]]] = {}
    
    def load(self) -> str:
        """llms.txt 파일을 동기적으로 로드"""
        with httpx.Client(timeout=30.0) as client:
            response = client.get(self.llms_txt_url)
            response.raise_for_status()
            self._content = response.text
            self._parse_index()
            return self._content
    
    async def aload(self) -> str:
        """llms.txt 파일을 비동기적으로 로드"""
        async with httpx.AsyncClient(timeout=30.0) as client:
            response = await client.get(self.llms_txt_url)
            response.raise_for_status()
            self._content = response.text
            self._parse_index()
            return self._content
    
    def _parse_index(self) -> None:
        """llms.txt 내용을 파싱하여 인덱스 생성"""
        if not self._content:
            return
        
        current_section = "general"
        self._index = {}
        
        for line in self._content.split('\n'):
            line = line.strip()
            
            # 섹션 헤더 감지
            if line.startswith('## '):
                current_section = line[3:].strip()
                if current_section not in self._index:
                    self._index[current_section] = []
            elif line.startswith('# '):
                current_section = line[2:].strip()
                if current_section not in self._index:
                    self._index[current_section] = []
            
            # 링크 추출: - [title](url) 또는 - [title](url): description
            link_match = re.match(r'-\s*\[([^\]]+)\]\(([^)]+)\)(?::\s*(.*))?', line)
            if link_match:
                title, url, description = link_match.groups()
                if current_section not in self._index:
                    self._index[current_section] = []
                self._index[current_section].append({
                    'title': title,
                    'url': url,
                    'description': description or ''
                })
    
    @property
    def content(self) -> str:
        """로드된 llms.txt 원본 내용"""
        if self._content is None:
            self.load()
        return self._content
    
    @property
    def index(self) -> dict[str, list[dict[str, str]]]:
        """파싱된 인덱스 (섹션별 링크 목록)"""
        if not self._index and self._content is None:
            self.load()
        return self._index
    
    def get_summary(self) -> str:
        """인덱스 요약 반환 (시스템 프롬프트용)"""
        if not self._index and self._content is None:
            self.load()
        
        lines = ["# API 문서 인덱스\n"]
        for section, items in self._index.items():
            lines.append(f"\n## {section}")
            for item in items:
                desc = f" - {item['description']}" if item['description'] else ""
                lines.append(f"- {item['title']}{desc}")
        
        return '\n'.join(lines)
    
    def search(self, keyword: str) -> list[dict[str, str]]:
        """키워드로 인덱스 검색"""
        if not self._index and self._content is None:
            self.load()
        
        results = []
        keyword_lower = keyword.lower()
        
        for section, items in self._index.items():
            for item in items:
                if (keyword_lower in item['title'].lower() or 
                    keyword_lower in item['description'].lower() or
                    keyword_lower in section.lower()):
                    results.append({
                        'section': section,
                        **item
                    })
        
        return results


# ==============================================================================
# 문서 fetcher
# ==============================================================================

class DocumentFetcher:
    """URL에서 문서를 가져오는 클래스"""
    
    def __init__(self, base_url: str = ""):
        self.base_url = base_url
        self._cache: dict[str, str] = {}
    
    def fetch(self, url: str) -> str:
        """URL에서 문서를 동기적으로 가져옴"""
        # 상대 URL 처리
        if not url.startswith('http'):
            url = urljoin(self.base_url, url)
        
        # 캐시 확인
        if url in self._cache:
            return self._cache[url]
        
        with httpx.Client(timeout=30.0, follow_redirects=True) as client:
            response = client.get(url)
            response.raise_for_status()
            content = response.text
            self._cache[url] = content
            return content
    
    async def afetch(self, url: str) -> str:
        """URL에서 문서를 비동기적으로 가져옴"""
        # 상대 URL 처리
        if not url.startswith('http'):
            url = urljoin(self.base_url, url)
        
        # 캐시 확인
        if url in self._cache:
            return self._cache[url]
        
        async with httpx.AsyncClient(timeout=30.0, follow_redirects=True) as client:
            response = await client.get(url)
            response.raise_for_status()
            content = response.text
            self._cache[url] = content
            return content
    
    @staticmethod
    def extract_links(content: str) -> list[dict[str, str]]:
        """마크다운 콘텐츠에서 링크 추출"""
        links = []
        # 마크다운 링크 패턴: [text](url)
        pattern = r'\[([^\]]+)\]\(([^)]+)\)'
        for match in re.finditer(pattern, content):
            title, url = match.groups()
            # 내부 앵커 링크 제외
            if not url.startswith('#'):
                links.append({'title': title, 'url': url})
        return links


# ==============================================================================
# Tool 정의
# ==============================================================================

def create_llms_txt_tools(llms_txt_url: str):
    """llms.txt 기반 tools 생성"""
    
    # 공유 인스턴스
    loader = LLMSTxtLoader(llms_txt_url)
    fetcher = DocumentFetcher(base_url=llms_txt_url.rsplit('/', 1)[0] + '/')
    
    # 초기 로드
    loader.load()
    
    @tool
    def search_docs(keyword: str) -> str:
        """인덱스에서 키워드로 문서를 검색합니다.
        
        Args:
            keyword: 검색할 키워드 (한글 또는 영어)
        
        Returns:
            검색 결과 목록 (문서 제목, URL, 설명)
        """
        results = loader.search(keyword)
        
        if not results:
            return f"'{keyword}'에 대한 검색 결과가 없습니다. 다른 키워드로 시도해보세요."
        
        lines = [f"'{keyword}' 검색 결과 ({len(results)}건):\n"]
        for r in results:
            desc = f" - {r['description']}" if r['description'] else ""
            lines.append(f"- [{r['title']}]({r['url']}){desc}")
            lines.append(f"  섹션: {r['section']}")
        
        return '\n'.join(lines)
    
    @tool
    def fetch_document(url: str) -> str:
        """URL에서 문서 내용을 가져옵니다. 상세 정보가 필요할 때 사용하세요.
        
        Args:
            url: 가져올 문서의 URL (search_docs 결과에서 얻은 URL)
        
        Returns:
            문서 내용 (마크다운 형식)
        """
        try:
            content = fetcher.fetch(url)
            
            # 너무 긴 경우 자르기 (토큰 절약)
            if len(content) > 15000:
                content = content[:15000] + "\n\n... (문서가 너무 깁니다. 필요한 부분만 확인하세요)"
            
            # 추가 링크가 있으면 알려주기
            links = fetcher.extract_links(content)
            if links:
                content += f"\n\n---\n📎 이 문서에서 발견된 링크 ({len(links)}개):\n"
                for link in links[:10]:  # 최대 10개만 표시
                    content += f"- [{link['title']}]({link['url']})\n"
                if len(links) > 10:
                    content += f"... 외 {len(links) - 10}개 더"
            
            return content
        except Exception as e:
            return f"문서를 가져오는 중 오류 발생: {str(e)}"
    
    @tool
    def get_doc_index() -> str:
        """전체 문서 인덱스를 반환합니다. 어떤 문서들이 있는지 확인할 때 사용하세요.
        
        Returns:
            전체 문서 인덱스 (섹션별 목록)
        """
        return loader.get_summary()
    
    return [search_docs, fetch_document, get_doc_index], loader


# ==============================================================================
# System Prompt 생성
# ==============================================================================

def create_system_prompt(loader: LLMSTxtLoader) -> str:
    """llms.txt 인덱스를 포함한 시스템 프롬프트 생성"""
    
    index_summary = loader.get_summary()
    
    return f"""당신은 llms.txt 기반 API 문서 가이드 전문가 챗봇입니다.

## 역할
사용자가 API에 대해 질문하면, 문서 인덱스를 검색하고 필요한 상세 문서를 가져와 정확한 답변을 제공합니다.

## Tool 사용 가이드

### 1. search_docs
- 키워드로 문서를 검색할 때 사용
- 한글/영어 모두 가능
- 예: "인증", "조회", "JWT", "API"

### 2. fetch_document  
- 검색 결과에서 얻은 URL로 상세 문서를 가져올 때 사용
- 문서 안에 추가 링크가 있으면 그것도 가져올 수 있음

### 3. get_doc_index
- 전체 문서 목록이 필요할 때 사용

## 답변 원칙
1. **정확성**: 문서 내용을 기반으로 답변
2. **간결성**: 핵심만 전달
3. **실용성**: API 사용 예시 포함 시 도움됨

## 문서 인덱스 (참고용)
{index_summary}

## 주의사항
- 문서에 없는 내용은 추측하지 마세요
- 필요 시 여러 문서를 참조하세요
- 링크 안의 링크도 확인이 필요할 수 있습니다
"""


# ==============================================================================
# Agent 생성
# ==============================================================================

def create_llms_txt_agent(
    llms_txt_url: str = "https://example.com/llms.txt",  # TODO: 실제 llms.txt URL로 변경하세요
    model_name: str = "databricks-gpt-5-2",
    system_prompt: Optional[str] = None,
):
    """llms.txt 기반 Langchain Agent 생성 (동기 버전)
    
    Args:
        llms_txt_url: llms.txt 파일 URL
        model_name: Databricks Foundation Model 엔드포인트 이름
        system_prompt: 커스텀 시스템 프롬프트 (None이면 자동 생성)
    
    Returns:
        Compiled Langchain Agent
    """
    
    # 모델 설정
    model = ChatDatabricks(endpoint=model_name)
    
    # Tools 및 Loader 생성
    tools, loader = create_llms_txt_tools(llms_txt_url)
    
    # 시스템 프롬프트 생성 (인덱스 포함)
    final_prompt = system_prompt or create_system_prompt(loader)
    
    # Agent 생성
    return create_agent(
        model=model,
        tools=tools,
        system_prompt=final_prompt
    )


@lru_cache(maxsize=1)
def get_default_llms_txt_agent():
    """기본 Agent 캐싱 (싱글톤)"""
    return create_llms_txt_agent()


In [0]:
%%writefile langchain_sync_wrapper_agent.py
"""
Langchain Agent Wrapper (동기 버전)

MLflow ResponsesAgent를 래핑하여 Langchain Agent를 
MLflow의 predict/predict_stream 인터페이스와 호환되도록 합니다.

이 모듈은 Langchain의 동기 방식 표준 예제입니다.
stream_mode="messages"를 사용하여 토큰 단위 스트리밍을 지원합니다.
"""
from typing import Any, Generator
import json

import mlflow

from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    to_chat_completions_input,
    output_to_responses_items_stream,
)

from langchain_core.messages.ai import AIMessageChunk
from langchain_core.messages.tool import ToolMessage

from langchain_sync_agent import create_llms_txt_agent
from uuid import uuid4


class LangchainSyncResponsesAgent(ResponsesAgent):
    """Langchain Agent - 동기 버전 (토큰 스트리밍 지원)"""

    def __init__(self):
        self.agent = create_llms_txt_agent()

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """전체 응답을 한번에 반환 (동기)
        
        predict_stream을 호출하여 모든 스트림 이벤트를 수집한 후 반환합니다.
        """
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done" or event.type == "error"
        ]
        return ResponsesAgentResponse(
            output=outputs, 
            custom_outputs=request.custom_inputs
        )

    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """동기 스트리밍 - stream_mode='messages'로 토큰 단위 스트리밍
        
        Langchain의 동기 방식 표준 패턴입니다.
        stream_mode="messages"는 튜플 (message_chunk, metadata) 형식으로 반환합니다.
        """
        
        # OpenAI 형식 메시지를 LangChain 형식으로 변환
        cc_msgs = to_chat_completions_input([i.model_dump() for i in request.input])
        
        # stream_mode="messages"로 토큰 단위 스트리밍
        # 반환 형식: (message_chunk, metadata) 튜플
        for msg, metadata in self.agent.stream(
            {"messages": cc_msgs}, 
            stream_mode="messages"
        ):
            try:
                if isinstance(msg, AIMessageChunk):
                    # 텍스트 콘텐츠가 있고, tool_calls가 없을 때 (최종 답변)
                    if msg.content and not msg.tool_calls:
                        yield ResponsesAgentStreamEvent(
                            **self.create_text_delta(
                                delta=msg.content, 
                                item_id=msg.id or str(uuid4())
                            ),
                        )
                
                elif isinstance(msg, ToolMessage):
                    # Tool 결과 스트리밍
                    for item in output_to_responses_items_stream([msg]):
                        yield item
                        
            except Exception as e:
                print(f"DEBUG: Error in stream handling: {e}")
                print(f"DEBUG: msg type: {type(msg)}, metadata: {metadata}")


# ==============================================================================
# MLflow 모델 등록
# ==============================================================================

mlflow.langchain.autolog()
AGENT = LangchainSyncResponsesAgent()
mlflow.models.set_model(AGENT)

## Agent 테스트

Agent와 상호작용하여 출력을 테스트합니다. `ResponsesAgent` 내에서 메서드를 수동으로 추적했기 때문에, Agent가 수행하는 각 단계의 트레이스를 확인할 수 있습니다. Langchain을 통한 LLM 호출은 autologging에 의해 자동으로 추적됩니다.

아래의 예시 입력을 실제 도메인에 맞는 예시로 변경하여 테스트하세요.

In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_sync_wrapper_agent import AGENT

# 단일 응답 테스트 (비스트리밍)
AGENT.predict(
    {
        "input": [{"role": "user", "content": "사용 가능한 API 목록을 알려주세요."}], 
        "custom_inputs": {"session_id": "test-session-123"}
    },
)

In [0]:
from langchain_sync_wrapper_agent import AGENT

# 스트리밍 응답 테스트 (상세 출력)
for chunk in AGENT.predict_stream(
    {
        "input": [{"role": "user", "content": "인증 방법을 알려주세요."}], 
        "custom_inputs": {"session_id": "test-session-123"}
    },
):
    if chunk.type != 'response.output_item.done':
        print("===============\n")
    print(f"{chunk=}\n")
    print(chunk.model_dump(exclude_none=True))

### Agent를 MLflow 모델로 로깅

배포 시 자동 인증 패스스루를 위해 Databricks 리소스를 지정합니다.

- **TODO**: Unity Catalog 함수가 [Vector Search 인덱스](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html)를 쿼리하거나 [외부 함수](https://docs.databricks.com/generative-ai/agent-framework/external-connection-tools.html)를 활용하는 경우, 해당 Vector Search 인덱스와 UC 연결 객체를 리소스로 포함해야 합니다. 자세한 내용은 [공식 문서](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough)를 참고하세요.

Agent 코드를 Python 파일에서 로깅합니다. [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code) 참고.

In [0]:
# 배포 시 자동 인증 패스스루를 위한 Databricks 리소스 설정
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint, DatabricksUCConnection
from pkg_resources import get_distribution

# TODO: 환경에 맞게 아래 설정값을 수정하세요
LLM_ENDPOINT_NAME = "databricks-gpt-5-2"  # 사용할 LLM 엔드포인트

resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
]

# 입력 예시 (모델 시그니처 추론용)
input_example = {
    "input": [
        {
            "role": "user",
            "content": "사용 가능한 API 목록을 알려주세요."
        }
    ],
    "custom_inputs": {
        "session_id": "test-session-123"
    }
}

# MLflow에 Agent 로깅
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="langchain_sync_agent",
        python_model="langchain_sync_wrapper_agent.py",
        input_example=input_example,
        pip_requirements=[
            "databricks-langchain",
            "backoff",
            f"databricks-connect=={get_distribution('databricks-connect').version}",
        ],
        code_paths=[
            "langchain_sync_agent.py",
        ],
        resources=resources,
    )

## [Agent Evaluation](https://docs.databricks.com/mlflow3/genai/eval-monitor)으로 Agent 평가

평가 데이터셋의 요청이나 예상 응답을 수정하고, Agent를 반복 개선하면서 평가를 실행할 수 있습니다. MLflow를 활용하여 계산된 품질 메트릭을 추적합니다.

[사전 정의된 LLM 스코어러](https://docs.databricks.com/mlflow3/genai/eval-monitor/predefined-judge-scorers)를 사용하거나, [커스텀 메트릭](https://docs.databricks.com/mlflow3/genai/eval-monitor/custom-scorers)을 추가하여 Agent를 평가하세요.

In [0]:
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness

# 평가 데이터셋 정의
# TODO: 실제 사용 사례에 맞게 질문과 예상 응답을 수정하세요
eval_dataset = [
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "사용 가능한 API 목록을 알려주세요."
                }
            ]
        },
        "expected_response": None
    },
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "인증 방법을 알려주세요."
                }
            ]
        },
        "expected_response": None
    }
]

# Agent 평가 실행
eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda input: AGENT.predict({"input": input, "custom_inputs": {"session_id": "evaluation-session"}}),
    scorers=[RelevanceToQuery(), Safety()],  # 필요에 따라 스코어러 추가
)

# MLflow UI에서 평가 결과 확인 (콘솔 출력 참조)

## 배포 전 Agent 검증

Agent를 등록하고 배포하기 전에 [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API를 통해 사전 배포 검증을 수행합니다. 자세한 내용은 [공식 문서](https://docs.databricks.com/machine-learning/model-serving/model-serving-debug.html#validate-inputs)를 참고하세요.

In [0]:
# 배포 전 검증 - 로깅된 모델이 정상적으로 예측을 수행하는지 확인
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/langchain_sync_agent",
    input_data={
        "input": [{"role": "user", "content": "사용 가능한 API 목록을 알려주세요."}], 
        "custom_inputs": {"session_id": "validation-session"}
    },
    env_manager="uv",
)

## Unity Catalog에 모델 등록

아래의 `catalog`, `schema`, `model_name`을 업데이트하여 MLflow 모델을 Unity Catalog에 등록합니다.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: Unity Catalog 모델의 catalog, schema, model_name을 환경에 맞게 수정하세요
catalog = "ml"
schema = "default"
model_name = "langchain-sync-agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# Unity Catalog에 모델 등록
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

## Agent 배포

Agent를 Databricks Model Serving 엔드포인트로 배포합니다.

In [0]:
from databricks import agents

# 참고: scale_to_zero=True를 전달하면 비용 절감을 위한 스케일 투 제로가 활성화됩니다.
# 프로덕션 워크로드에는 권장되지 않습니다 (스케일 제로 상태에서는 용량이 보장되지 않음).
# 스케일 제로 상태의 엔드포인트는 다시 스케일업하는 동안 응답 시간이 길어질 수 있습니다.
agents.deploy(
    UC_MODEL_NAME, 
    uc_registered_model_info.version, 
    tags={"endpointSource": "local_development"}
)

## 다음 단계

Agent가 배포되면 다음 작업을 수행할 수 있습니다:

- **AI Playground에서 테스트**: 추가 검증을 위해 대화형으로 Agent 테스트
- **피드백 수집**: 조직 내 전문가(SME)와 Agent 공유하여 피드백 수집
- **프로덕션 애플리케이션에 통합**: 실제 서비스에 Agent 임베딩

자세한 내용은 [공식 문서](https://docs.databricks.com/generative-ai/deploy-agent.html)를 참고하세요.